In [33]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import warnings

In [2]:
tmp_data_path = '../MA_data/data/tmp'
data_path = '../MA_data/data'

s_year = 1997
e_year = 2020

In [3]:
sdc_tnic = pd.read_pickle(tmp_data_path+f"/sdc_tnic_{s_year}_{e_year}")

In [4]:
with open(tmp_data_path+f"/tnic_info_3_pairs_{s_year-1}_{e_year-1}", 'rb') as f:
    gvkey_lsts, key_ind_maps , ind_key_maps = pickle.load(f)

# not allow single day merge multiple target

- unable to predict such condition


In [124]:
def same_day_only_one(sdc_tnic_raw):
    print(sdc_tnic_raw.shape)
    sdc_tnic_raw = sdc_tnic_raw.copy()
    sdc_tnic_one = sdc_tnic_raw.groupby(['AGVKEY', 'DA']).first().reset_index(drop=False)
    print(sdc_tnic_one.shape)
    sdc_tnic_one.sort_values(by = ['DA'], axis=0, inplace=True)
    return sdc_tnic_one

In [125]:
sdc_tnic_one = same_day_only_one(sdc_tnic)

(9661, 50)
(9448, 50)


In [127]:
sdc_tnic_one.columns

Index(['AGVKEY', 'DA', 'ACU', 'ASIC2', 'ABL', 'ANL', 'APUBC', 'AUP', 'AUPSIC',
       'AUPBL', 'AUPNAMES', 'AUPPUB', 'BLOCK', 'CREEP', 'DE', 'STATC', 'SYNOP',
       'VAL', 'PCTACQ', 'PSOUGHTOWN', 'PSOUGHT', 'PHDA', 'PCTOWN', 'PSOUGHTT',
       'PRIVATIZATION', 'DEAL_NO', 'TCU', 'TSIC2', 'TBL', 'TNL', 'TPUBC',
       'TUP', 'TUPSIC', 'TUPBL', 'TUPNAMES', 'TUPPUB', 'SIC_A', 'SIC_T',
       'YEAR', 'AS_PERMNO', 'AP_PERMNO', 'TS_PERMNO', 'TP_PERMNO', 'AS_GVKEY',
       'AP_GVKEY', 'TS_GVKEY', 'TP_GVKEY', 'GVKEY_STATUS', 'MA_TYPE',
       'TGVKEY'],
      dtype='object')

# obtain frequent acquirer

In [5]:
A_freq = pd.DataFrame(sdc_tnic.AGVKEY.value_counts()).reset_index(drop=False)
A_freq = A_freq[A_freq.AGVKEY >= 5]
A_freq.columns = ["GVKEY", "freq"]
print(f"totally {A_freq.shape[0]} numbers of frequent Acquirers")

totally 511 numbers of frequent Acquirers


In [37]:
A_freq

,GVKEY,freq
0,5047,101
1,2176,49
2,177088,41
3,11856,41
4,6268,39
...,...,...
506,5439,5
507,23220,5
508,164652,5
509,1831,5


# Preprocess Data
- preprcess data 
- this makes Dataloader very simple

In [147]:
def dataloader_preproceser(focal_gvkey):
    
    def get_arr_c(focal_gvkey):
    # part 1, get c  
        def get_focal_df(focal_gvkey):
            '''
            output: will be a df contains 3 columns: DATE, AGVKEY, EVENT_TYPE, SCORE
                DATE: datetime.dt object
                AGVKEY: str: 4 - 6 digits
                EVENT_TYPE: 1:self 0:peer (integer)
                SCORE: TNIC similarity last year for event type 0, otherwise 1
            
            Use DA!!!! not DE

            '''
            def helper1(row):
                if row.AGVKEY == focal_gvkey:
                    return 1 # integer 1
                else:
                    return 0 # integer 0   
            sdc_lst = []
            for focal_year in range(s_year-1, e_year):  
                with open(tmp_data_path+f"/a5_top_10_peers_tnic2_{focal_year}.pickle", 'rb') as f:
                    top_peers = pickle.load(f)
                try:
                    top_peers = top_peers[focal_gvkey] # a dataframe
         #           print(top_peers)
                    top_peers_lst = top_peers.gvkey2.tolist()
                    selected_sdc_tnic = sdc_tnic[ (sdc_tnic['AGVKEY'].isin(top_peers_lst + [focal_gvkey])) & (sdc_tnic.YEAR == focal_year+1) ] 
                    selected_sdc_tnic.reset_index(drop=True)
                    if selected_sdc_tnic.shape[0] > 0:
                        #print(selected_sdc_tnic[['DE', 'AGVKEY']] , top_peers[['gvkey2', 'score']])
                        df = selected_sdc_tnic[['DA', 'AGVKEY', 'TGVKEY']]


                        df['EVENT_TYPE'] = df.apply(helper1, axis=1)
                        #print(df)

                        score_df = top_peers[['gvkey2', 'score']]

                        df = df.merge(score_df, left_on='AGVKEY', right_on = 'gvkey2', how = 'left')
                        df = df[['DA','AGVKEY', 'EVENT_TYPE', 'score', 'TGVKEY']]
        #                print(df)
                        df = df.fillna(1)
                        df.columns = ['UPDATE_DATE','AGVKEY','EVENT_TYPE', 'SCORE', 'TGVKEY'] # rename
                        df = df.reset_index(drop=True)
                        sdc_lst.append(df)
                    #print(len(sdc_lst))
                except:
                    pass

            focal_df = pd.concat(sdc_lst, axis=0)
            focal_c = focal_df.reset_index(drop=True) 
            focal_c = focal_c.sort_values(by = ['UPDATE_DATE']) # date time is unsortable..
            focal_c.reset_index(drop=True, inplace=True)
            return focal_c

        def convert_date(df):
            def datetime_converter(date_time):
                base_time = np.datetime64('1997-01-01')
                days_diff = np.datetime64(date_time.date()) - base_time
                return days_diff.astype(int)
            for idx, row in df.iterrows():
                df.loc[idx, 'UPDATE_DATE'] = datetime_converter(df.loc[idx, 'UPDATE_DATE'])

            df.sort_values(by = ['UPDATE_DATE']).reset_index(drop=True, inplace=True)
            return df

        def making_time_diff(focal_c2):
            '''
            df = focal_c; update date is the integer form that count the date from base_date (1997 01 01)

            WARNING: the No.1 event set time-diff = 0
            '''
            tmp_columns = focal_c2.columns.tolist()
            focal_c2['UPDATE_DATE'] = [0] + [1 if timediff==0 else timediff for timediff in focal_c2.UPDATE_DATE.diff().tolist()[1:] ]
            focal_c2.columns = ['time_diff'] + tmp_columns[1:]
            return focal_c2

        def __main__():
            focal_c = get_focal_df(focal_gvkey)
            focal_c2 = convert_date(focal_c.copy())
            focal_c3 = making_time_diff(focal_c2.copy())
            arr_c = np.array(focal_c3[['time_diff', 'EVENT_TYPE', 'SCORE']])
            return arr_c, focal_c

        arr, focal_c = __main__()

        return arr, focal_c

    def get_arr_b(focal_c, focal_gvkey):
        def add_datetime(df):
            def helper(row):
                return np.datetime64(str(row.year+1)+'-01-01')
            df['UPDATE_DATE'] = df.apply(helper, axis=1)
            return df
        def obtain_fv(focal_gvkey, focal_c, fv):
            year_min, year_max = min([date.year for date in focal_c.UPDATE_DATE.tolist()]), max([date.year for date in focal_c.UPDATE_DATE.tolist()])
            fv_subset = fv[(fv.year >= year_min-1) & (fv.year <= year_max-1) & (fv.gvkey == focal_gvkey)]
            fv_subset = fv_subset[['gvkey', 'year','UPDATE_DATE', 'at', 'sale', 'ch', 'm2b', 'lev', 'roa', 'ppe',
               'cash2asset', 'cash2sale', 'sale2asset', 'de', 'roe', 'd_sale', 'd_at']]
            fv_subset.columns=['AGVKEY', 'year','UPDATE_DATE', 'at', 'sale', 'ch', 'm2b', 'lev', 'roa', 'ppe',
               'cash2asset', 'cash2sale', 'sale2asset', 'de', 'roe', 'd_sale', 'd_at']
            return fv_subset

        def __main__():
            with open(tmp_data_path+"/afreq_full_fv.pickle", "rb") as f:
                fv = pickle.load(f)
            fv = add_datetime(fv)
            focal_b = obtain_fv("5047", focal_c, fv)
            arr_b = np.array(focal_b.iloc[:, 3:])
            return arr_b, focal_b

        arr = __main__()
        return arr

    def create_main_timeline(focal_b, focal_c):
        '''
        WARNING: GLOBAL and LOCAL time both start from 0!

        '''
        def helper(row):
            if (row.EVENT_TYPE == 1) or (row.EVENT_TYPE == 0):
                return 'past'
            else:
                return "fv"

        def helper2(row):
            if row.EVENT_TYPE == 1:
                return "1"
            elif row.EVENT_TYPE == 0:
                return "2"
            else:
                return "3"

        tmp = pd.concat([focal_c, focal_b]).sort_values(by=['UPDATE_DATE'])
        tmp['EVENT_TYPE_countcreater'] = tmp.apply(helper, axis=1)
        tmp['EVENT_TYPE_true'] = tmp.apply(helper2, axis=1)
        tmp['LOCAL_IDX'] = tmp.groupby(['EVENT_TYPE_countcreater'])['UPDATE_DATE'].rank(ascending=True) -1 # rank start with 1
        ## with local_idx using rank, local idx is not continuous (may have gap)
        
        tmp_columns = tmp.columns
        tmp.reset_index(drop=True, inplace=True)
        tmp.reset_index(drop=False, inplace=True)

        tmp.columns = ['GLOBAL_IDX']+ tmp_columns.tolist() # rename global index

        tmp = tmp[['GLOBAL_IDX', 'LOCAL_IDX', 'UPDATE_DATE', 'EVENT_TYPE_true', 'TGVKEY']]

        tmp.columns = ['GLOBAL_IDX', 'LOCAL_IDX', 'UPDATE_DATE', 'EVENT_TYPE', 'TGVKEY'] # rename



        return tmp
    
    def __main__():
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            arr_c, focal_c = get_arr_c(focal_gvkey)
        arr_b, focal_b = get_arr_b(focal_c, focal_gvkey)
        timeline = create_main_timeline(focal_b, focal_c)
        return arr_c, arr_b, timeline
    
    arr_c, arr_b, timeline = __main__()
    return arr_c, arr_b, timeline

In [148]:

c, b, df = dataloader_preproceser('121817')
    

In [149]:
c.shape # t, I(1=self), s(1=self)

(45, 3)

In [150]:
b.shape # raw fv 

(21, 14)

- '1' = 'self'
- '2' = 'peer'
- '3' = 'fv'


In [151]:
with open(tmp_data_path+"/testing1.pickle", "wb") as f:
    pickle.dump((c,b,df), f)

# prepare

get all inputs of timing module from c, b, df

In [ ]:
arr_delta_time = 

In [ ]:
class MYDataset():
    def __init__(self):
        self.raw_fv = 
        
    
    def create_A_freq_table(sdc_tnic, thres):
        '''
        thres: the minimum number of self MA event across obervation window (here fixed 1997 to 2020)
        '''
        A_freq = pd.DataFrame(sdc_tnic.AGVKEY.value_counts()).reset_index(drop=False)
        A_freq = A_freq[A_freq.AGVKEY >= 5]
        A_freq.columns = ["GVKEY", "freq"]
        print(f"totally {A_freq.shape[0]} numbers of frequent Acquirers")
    
    def read_raw_fv(path, s_year, e_year):
        

# Create dataset

- 4 things
    - list of b
    - list of c (idx must be corresponding)
    - list of timeline
    - map
    

In [97]:

with open(data_path+"/freq_acquirer_lst.pickle", 'wb') as f:
    pickle.dump(A_freq.GVKEY.values.tolist(), f)

In [38]:
A_freq

,GVKEY,freq
0,5047,101
1,2176,49
2,177088,41
3,11856,41
4,6268,39
...,...,...
506,5439,5
507,23220,5
508,164652,5
509,1831,5


In [39]:
A_freq_lst = A_freq.GVKEY.tolist()

In [105]:
arr_cs = []
arr_bs = []
timelines = []
idx_to_gvkey = {}

for i, gvkey in enumerate(tqdm(A_freq_lst)):
    arr_c, arr_b, timeline = dataloader_preproceser(gvkey)
    
    arr_cs.append(arr_c)
    arr_bs.append(arr_b)
    timelines.append(timeline)
    
    idx_to_gvkey[i] = gvkey
    
assert len(arr_cs) == len(arr_bs) == len(timelines) == len(A_freq_lst)
    
    
    

100%|██████████| 511/511 [23:19<00:00,  2.74s/it]


In [106]:
with open(data_path+"/dataset_top10_freq5_1997_2020.pickle", "wb") as f:
    pickle.dump((arr_cs, arr_bs, timelines), f)

In [57]:
timelines[0]

,GLOBAL_IDX,LOCAL_IDX,UPDATE_DATE,EVENT_TYPE
0,0,0,1997-01-01,3
1,1,0,1997-04-16,1
2,2,1,1997-05-30,1
3,3,2,1997-06-27,1
4,4,3,1997-07-01,2
...,...,...,...,...
215,215,194,2017-03-27,1
216,216,195,2017-10-31,2
217,217,21,2018-01-01,3
218,218,196,2018-07-18,1


In [56]:
timelines[0]["EVENT_TYPE"].value_counts()[1]


97

In [58]:
now = timelines[0]

In [61]:
sub = now[now["EVENT_TYPE"] == '1']

In [60]:
sub = now[now["EVENT_TYPE"] == '1']

,GLOBAL_IDX,LOCAL_IDX,UPDATE_DATE,EVENT_TYPE
1,1,0,1997-04-16,1
2,2,1,1997-05-30,1
3,3,2,1997-06-27,1
5,5,4,1997-07-01,1
7,7,6,1997-07-18,1
...,...,...,...,...
209,209,190,2015-05-29,1
213,213,193,2016-09-08,1
215,215,194,2017-03-27,1
218,218,196,2018-07-18,1


In [78]:
for i, row in sub.iterrows():
    print(now[now.UPDATE_DATE == pd.Timestamp(f'{row.UPDATE_DATE.year}-01-01 00:00:00')].LOCAL_IDX.values[0])
    break

0


In [73]:
pd.Timestamp(f'{1997}-01-01 00:00:00')

Timestamp('1997-01-01 00:00:00')

In [ ]:
row.UPDATE_DATE

In [79]:
now[now.UPDATE_DATE == pd.Timestamp(f'{row.UPDATE_DATE.year}-01-01 00:00:00')].LOCAL_IDX.values[0]

0

In [80]:
arr_c # time indicator tnic

array([[0.000e+00, 0.000e+00, 7.710e-02],
       [6.600e+01, 0.000e+00, 7.320e-02],
       [1.070e+02, 0.000e+00, 8.990e-02],
       [2.290e+02, 0.000e+00, 5.970e-02],
       [1.100e+02, 0.000e+00, 8.000e-02],
       [7.500e+01, 0.000e+00, 5.700e-02],
       [8.800e+01, 0.000e+00, 5.700e-02],
       [1.000e+01, 1.000e+00, 1.000e+00],
       [5.700e+01, 1.000e+00, 1.000e+00],
       [1.960e+02, 1.000e+00, 1.000e+00],
       [1.680e+02, 1.000e+00, 1.000e+00],
       [2.690e+02, 0.000e+00, 7.480e-02],
       [1.930e+02, 0.000e+00, 5.600e-02],
       [2.650e+02, 0.000e+00, 4.220e-02],
       [7.000e+02, 0.000e+00, 5.710e-02],
       [9.200e+01, 0.000e+00, 7.640e-02],
       [3.650e+02, 0.000e+00, 6.580e-02],
       [6.200e+01, 0.000e+00, 8.660e-02],
       [9.300e+01, 0.000e+00, 6.230e-02],
       [8.500e+01, 0.000e+00, 8.690e-02],
       [1.000e+01, 0.000e+00, 1.501e-01],
       [5.410e+02, 0.000e+00, 3.240e-02],
       [3.300e+02, 0.000e+00, 1.569e-01],
       [4.100e+01, 0.000e+00, 1.56

In [81]:
arr_b

array([[ 3.04012000e+05,  8.85400000e+04,  3.70030000e+04,
         1.64619304e+00,  4.75895688e-01,  2.69824875e-02,
         1.06298436e-01,  5.35767172e-02,  2.50350296e-01,
         2.91238504e-01,  4.20111505e+00,  2.38196179e-01,
         1.27309303e-01,  1.16041732e-01],
       [ 3.55935000e+05,  9.98200000e+04,  3.70030000e+04,
         1.80197843e+00,  4.91777993e-01,  2.61171281e-02,
         1.00383497e-01,  5.35767172e-02,  2.50350296e-01,
         2.80444463e-01,  4.50208333e+00,  2.39094650e-01,
         1.27400045e-01,  1.70792600e-01],
       [ 4.05200000e+05,  1.10832000e+05,  3.70030000e+04,
         2.12668918e+00,  4.97959033e-01,  2.64486673e-02,
         1.01238894e-01,  5.35767172e-02,  2.50350296e-01,
         2.73524186e-01,  4.74124116e+00,  2.51826961e-01,
         1.10318573e-01,  1.38410103e-01],
       [ 4.37006000e+05,  1.28051000e+05,  8.19500000e+03,
         1.95406822e+00,  4.60661867e-01,  2.91414763e-02,
         9.15662485e-02,  1.87526029e-02,  6.

In [99]:
1+1

2